In [42]:
import preprocessors

In [35]:
myLoader = preprocessors.Loader("C:/Users/emily/Documents/DissertationProject/training/training_setA_csv", ['HR', 'O2Sat', 'SBP', 'MAP', 'DBP', 'Resp', 'Temp'])
df = myLoader.ARIMA_VAR_preprocessor(max_patients=100, min_time_points=30)
y_train, y_pred = myLoader.split_train_test(df, 4)

Patient 0: Dropping — constant columns found
Patient 1: Less than 30 time points — skipping.
Patient 3: Less than 30 time points — skipping.
Patient 4: Dropping — constant columns found
Patient 5: Less than 30 time points — skipping.
Patient 9: Less than 30 time points — skipping.
Patient 10: Dropping — constant columns found
Patient 11: Less than 30 time points — skipping.
Patient 12: Dropping — constant columns found
Patient 14: Less than 30 time points — skipping.
Patient 15: Less than 30 time points — skipping.
Patient 16: Dropping — constant columns found
Patient 19: Less than 30 time points — skipping.
Patient 21: Less than 30 time points — skipping.
Patient 22: Less than 30 time points — skipping.
Patient 23: Dropping — constant columns found
Patient 24: Dropping — constant columns found
Patient 25: Less than 30 time points — skipping.
Patient 26: Less than 30 time points — skipping.
Patient 31: Dropping — constant columns found
Patient 32: Dropping — constant columns found
Pati

In [31]:
y_train

HR  O2Sat    SBP    MAP   DBP  Resp   Temp
Patient_ID ICULOS                                               
0          0       117.0   99.0  116.0   97.0  81.0  20.0  36.00
           1       117.0   99.0  116.0   97.0  81.0  20.0  36.00
           2       117.0   99.0  116.0   97.0  81.0  20.0  36.00
           3       117.0   99.0  116.0   97.0  81.0  20.0  36.00
           4       117.0   99.0  116.0   97.0  81.0  20.0  36.00
...                  ...    ...    ...    ...   ...   ...    ...
96         327      72.0   99.0  163.0   97.0  95.0  18.0  37.39
           328      74.0   99.0  142.0   97.0  95.0  18.0  37.67
           329      74.0   98.0  152.0   75.0  95.0  19.0  37.67
           330      54.0   98.0  167.0  114.0  95.0  24.0  37.67
           331      55.0   93.0  130.0   83.0  95.0  22.0  37.72

[12101 rows x 7 columns]

In [4]:
y_pred

HR  O2Sat    SBP   MAP   DBP  Resp   Temp
Patient_ID ICULOS                                              
0          44       72.0   99.0  158.0  98.0  68.0  26.0  37.06
           45       72.0   99.0  139.0  84.0  54.0  26.0  37.06
           46       76.0   98.0  138.0  82.0  53.0  27.0  37.11
           47       78.0   97.0  138.0  83.0  53.0  26.0  37.11
1          41      108.0   96.0   99.0  63.0  47.0  21.0  37.67
...                  ...    ...    ...   ...   ...   ...    ...
95         39       89.0   97.0  128.0  82.0  59.0  21.0  36.39
96         42       52.0   98.0  121.0  78.0  55.0  12.5  37.33
           43       52.0   98.0  115.0  74.0  53.0   9.0  37.33
           44       52.0   98.0  111.0  70.0  49.0  11.0  37.33
           45       51.0   99.0  112.0  71.0  51.0  11.0  37.33

[388 rows x 7 columns]

In [32]:
from statsmodels.tsa.stattools import adfuller

for col in y_train.columns:
    result = adfuller(y_train[col])
    
    print(f"{col}: p-value = {result[1]}")


HR: p-value = 4.885879185774651e-17
O2Sat: p-value = 3.7044665374306566e-22
SBP: p-value = 4.668991639998474e-17
MAP: p-value = 3.37891964908557e-19
DBP: p-value = 1.4207042111313727e-18
Resp: p-value = 3.8636821608730997e-19
Temp: p-value = 6.868580228528386e-22


In [38]:
from sktime.forecasting.var import VAR
model = VAR(maxlags=2, method="ols", verbose=False, trend="c", random_state=42)

In [39]:
from sktime.forecasting.base import ForecastingHorizon

model.fit(y_train)
fh = ForecastingHorizon([1, 2, 3, 4], is_relative=True)
forecasts = model.predict(fh)

In [40]:
forecasts

HR      O2Sat         SBP        MAP        DBP  \
Patient_ID ICULOS                                                            
0          44       70.862070  96.606026  157.424137  90.406187  66.518350   
           45       72.017566  96.317298  158.910113  94.720804  67.250587   
           46       73.806358  96.417051  155.880152  91.899776  65.550761   
           47       75.167536  96.795875  152.375212  89.790486  63.889030   
1          41      114.962771  95.330566  120.420020  77.046488  60.547877   
...                       ...        ...         ...        ...        ...   
95         39       97.861972  94.923259  134.879332  73.246784  50.416845   
96         42       51.904165  99.574414  111.460142  71.652546  51.378877   
           43       52.478270  99.004143  109.313918  71.538120  51.436881   
           44       53.659174  99.055318  108.509953  70.462749  50.314254   
           45       53.473427  99.256590  112.129461  72.187269  51.088954   

                        Resp       Temp  
Patient_ID ICULOS                        
0          44      23.382905  37.260062  
           45      23.742392  37.458839  
           46      26.959493  38.118423  
           47      27.697126  38.359054  
1          41      21.363237  37.715698  
...                      ...        ...  
95         39      30.909908  36.574575  
96         42       8.972587  37.373600  
           43      10.294684  37.213147  
           44      11.060529  37.051725  
           45      11.408650  37.018478  

[388 rows x 7 columns]

In [44]:
import evaluators

myEvaluator = evaluators.Evaluator(y_pred, forecasts)
myEvaluator.sktime_dtw()
myEvaluator.sktime_ddtw()
myEvaluator.tslearn_dtw()

Mean sktime DTW distance for feature HR: 430.6348299632063
Mean sktime DTW distance for feature O2Sat: 26.052426743937513
Mean sktime DTW distance for feature SBP: 1470.487551996165
Mean sktime DTW distance for feature MAP: 665.5541113093424
Mean sktime DTW distance for feature DBP: 406.39751812593914
Mean sktime DTW distance for feature Resp: 75.4094156327051
Mean sktime DTW distance for feature Temp: 1.0890900583583811
Mean sktime DDTW distance for feature HR: 70.45827378496547
Mean sktime DDTW distance for feature O2Sat: 5.119314403816615
Mean sktime DDTW distance for feature SBP: 366.73642064963514
Mean sktime DDTW distance for feature MAP: 178.2568025041841
Mean sktime DDTW distance for feature DBP: 116.4563631465721
Mean sktime DDTW distance for feature Resp: 23.702112021211086
Mean sktime DDTW distance for feature Temp: 0.11503470668182936
Mean tslearn DTW distance: 47.05199057386343


In [12]:
from sktime.forecasting.compose import TransformedTargetForecaster
from sktime.forecasting.var import VAR
from sktime.transformations.series.difference import Differencer

model = TransformedTargetForecaster(steps=[
    ("diff", Differencer()),     # makes the data stationary
    ("var", VAR())               # fit VAR on differenced data
])
